In [1]:
import numpy as np
import pandas as pd 
import csv 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")

In [2]:
#  loading the data set 
file = pd.read_csv('BSOM_DataSet_for_HW3.csv')


# to check whether there are any empty values or not 
df=file[['all_mcqs_avg_n20', 'all_NBME_avg_n4', 'CBSE_01',  'CBSE_02', 'LEVEL']]


df['LEVEL'].fillna(value='A', inplace=True) 

df[['CBSE_02']].fillna(method ='bfill')
#.fillna(52, inplace=True)
df.fillna(df['CBSE_02'].mean(), inplace=True)

df.isnull().any()


all_mcqs_avg_n20    False
all_NBME_avg_n4     False
CBSE_01             False
CBSE_02             False
LEVEL               False
dtype: bool

In [3]:
LEVEL = {'A': 0,'B': 1,'C':2,'D':3}

df.LEVEL = [LEVEL[item] for item in df.LEVEL] 
print(df) 

     all_mcqs_avg_n20  all_NBME_avg_n4  CBSE_01  CBSE_02  LEVEL
0               0.736           0.7700       42     68.0      1
1               0.740           0.8000       44     67.0      2
2               0.807           0.8125       41     78.0      0
3               0.886           0.9250       68     91.0      0
4               0.839           0.8550       57     74.0      1
..                ...              ...      ...      ...    ...
110             0.637           0.6825       45     52.0      2
111             0.751           0.8150       53     60.0      2
112             0.811           0.8325       50     77.0      1
113             0.729           0.7275       43     68.0      2
114             0.839           0.8625       54     73.0      1

[115 rows x 5 columns]


In [4]:



from sklearn.compose import ColumnTransformer

#onhtencder = OneHotEncoder(categorical_features = [0]) 

onhtencder= ColumnTransformer([("LEVEL", OneHotEncoder(), [0])], remainder = 'passthrough')

data = onhtencder.fit_transform(df[['LEVEL']]).toarray() 
#data

In [5]:

X= df.iloc[:,:]

X_norm = (X.iloc[:,:-1] - X.iloc[:,:-1].mean()) / (X.iloc[:,:-1].max() - X.iloc[:,:-1].min())



X_modified = pd.concat([X_norm, X.iloc[:,-1]],axis=1)

target=df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X_modified, data, test_size=0.20,random_state=9)

In [6]:
def Weights(ttl_weits,ip,hidn,otpt):
    w1 = np.reshape(ttl_weits[:hidn*(ip+1)], (hidn, ip+1), 'F')
    w2 = np.reshape(ttl_weits[hidn*(ip+1):], (otpt, hidn+1), 'F')
    return w1,w2

    

In [7]:


# SIG function to normalize inputs
def SIG(z):
    return 1 / (1 + np.exp(-z))

# SIG derivatives to adjust synaptic weights
def SIGGrad(z):
    return np.multiply(SIG(z), 1-SIG(z))




In [8]:
def Cost(h,y):
    cost = np.sum(np.multiply(y, np.log(h))+np.multiply(1-y, np.log(1-h)))
    return cost

In [9]:
Train_ip=np.array(X_train.iloc[:,:-1])
Train_otpts=np.array(y_train)


In [10]:


np.random.seed(22)

wt1 =  np.random.random((1,5))#- 1    
wt2 =  np.random.random((4,2))    
ttl_weits = np.hstack((wt1.ravel(order='F'), wt2.ravel(order='F')))    

# layers configuration 

ip_lyr = 4
hd_lyr = 1
no_labls = 4

print('wt1',wt1)
print('wt2',wt2)
print('ttl_weits',ttl_weits)
print(ttl_weits.shape)

wt1 [[0.20846054 0.48168106 0.42053804 0.859182   0.17116155]]
wt2 [[0.33886396 0.27053283]
 [0.69104135 0.22040452]
 [0.81195092 0.01052687]
 [0.5612037  0.81372619]]
ttl_weits [0.20846054 0.48168106 0.42053804 0.859182   0.17116155 0.33886396
 0.69104135 0.81195092 0.5612037  0.27053283 0.22040452 0.01052687
 0.81372619]
(13,)


In [11]:
def Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, X, y):
    wt1 ,wt2= Weights(ttl_weits,ip_lyr,hd_lyr,no_labls)

    
    l = len(y)
   
    ones = np.ones((l,1))
    #print('ones',ones)
    a1 = np.hstack((ones, X))
    #print('a1',a1)
    a2 = SIG(a1 @ wt1.T)
    #print('a2',a2)
    a2 = np.hstack((ones, a2))
    #print('a2',a2)
    h = SIG(a2 @ wt2.T)
    #print('h',h)

   
    
    c=Cost(h,Train_otpts)
    
    return c / (-l)    
    

In [12]:
cf_er = Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, Train_ip,Train_otpts)


In [13]:
def NN(ttl_weits, ip_lyr, hd_lyr, no_labls, X, y):
    
    initt_wt1,initt_wt2 = Weights(ttl_weits,ip_lyr,hd_lyr,no_labls)
    delta1 = np.zeros(initt_wt1.shape)
    #print('delta1',delta1)
    #print('delta1 shape',delta1.shape)
    delta2 = np.zeros(initt_wt2.shape)
    #print('delta2',delta2)
    #print('delta2 shape',delta2.shape)
    lenth = len(y)

    ones = np.ones((lenth,1))
    
    
    a1 = np.hstack((ones, X))
    #print('a1',a1)
    #print('al shape',a1.shape)
    z2 = a1 @ initt_wt1.T
    #print('z2',z2)
    #print('z2 sha',z2.shape)
    a2 = np.hstack((ones, SIG(z2)))
    #print('a2 ',a2)
    #print('a2 ',a2.shape)
    z3 = a2 @ initt_wt2.T
    #print('z3',z3)
    #print('z3 ',z3.shape)
    a3 = SIG(z3)
    #print('a3',a3)
    #print('a3',a3.shape)
    y1=pd.DataFrame(y)

    d3 = a3 - y
    #print('d3',d3)
    #print('d3',d3.shape)
    z2 = np.hstack((ones, z2))
    #print('z2',z2)
    #print('z2',z2.shape)
    d2 = np.multiply(initt_wt2.T @ d3.T, SIGGrad(z2).T)
    #print('d2',d2)
    #print('d2',d2.shape)
    delta1 = delta1 + d2[1:,:] @ a1
    #print('delta1',delta1)
    #print('delta1',delta1.shape)
    delta2 = delta2 + d3.T @ a2
    #print('delta2',delta2)
    #print('delta2 ',delta2.shape)
        
        
    delta1 /= lenth
    delta2 /= lenth
   
    return np.hstack((delta1.ravel(order='F'), delta2.ravel(order='F')))

In [14]:
bckprop_wts = NN(ttl_weits, ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts)

#bckprop_wts

In [15]:
def predict(wt1, wt2, X, y):
    l = len(y)
    ones = np.ones((l,1))
    a1 = np.hstack((ones, X))
    a2 = SIG(a1 @ wt1.T)
    a2 = np.hstack((ones, a2))
    h = SIG(a2 @ wt2.T)
    return np.argmax(h, axis = 1) 

In [16]:

def checkGradient(ttl_weits,bckprop_wts,iplyr, hdlyr, no_labls,Train_ip,Train_otpts):
    eps = 0.22
   
    
    for i in range(5):
        iterattion = int(np.random.rand()*len(ttl_weits))
        #print('iterattion',iterattion)
        adding_eps = np.zeros((len(ttl_weits),1))
        #print('adding_eps',adding_eps)
        adding_eps[iterattion] = eps
        #print('adding_eps[iterattion]',adding_eps[iterattion])

        ThetaPlus = Feedfrwd(ttl_weits + adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts)
        
        #print('ThetaPlus',ThetaPlus)
        
        ThetaMinus  = Feedfrwd(ttl_weits - adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts)
        #print('ThetaMinus',ThetaMinus)
        
        num_app = (ThetaPlus - ThetaMinus) / float(2*eps)
        print('num_app',num_app)
       
        print('bckprop_wts[iterattion]',bckprop_wts[iterattion])
        
        



In [17]:
grad_chk=checkGradient(ttl_weits,bckprop_wts,ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts)

grad_chk

num_app 0.14451221291702812
bckprop_wts[iterattion] 0.1445916205321833
num_app -0.0033283890953048826
bckprop_wts[iterattion] -0.0033315205008140917
num_app 0.1725310969982559
bckprop_wts[iterattion] 0.17318609441646016
num_app 0.17229684529790548
bckprop_wts[iterattion] 0.17241141769797821
num_app 0.3789769973031507
bckprop_wts[iterattion] 0.37910457943231274


In [18]:
import scipy.optimize as opt
Final_wts = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts))


Final_wts1,Final_wts2= Weights(Final_wts, ip_lyr, hd_lyr, no_labls)

         Current function value: 1.349173
         Iterations: 100
         Function evaluations: 287
         Gradient evaluations: 287


In [19]:
Final_wts

array([-2.63666748e+00,  3.04452619e+00,  4.97599262e+00, -2.75562449e+00,
        9.27669942e+00, -3.27582789e+00,  1.58165312e-02,  3.88281267e-01,
       -2.28514774e+00,  9.74023178e+00, -2.54296509e+00, -3.13574078e+01,
       -1.48847583e+01])

## Predicting y values for test_data

In [20]:
pred = predict(Final_wts1,Final_wts2, X_test.iloc[:,:-1], y_test)
pred


array([2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0], dtype=int64)

In [21]:
y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_actual,pred))




[[4 0 1 0]
 [4 7 0 0]
 [0 4 2 0]
 [0 1 0 0]]


In [22]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


print('f1_score',f1_score(y_actual,pred, average="weighted"))
print('precision_score',precision_score(y_actual,pred, average="weighted"))
print('recall_score',recall_score(y_actual,pred, average="weighted"))  

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_actual,pred, pos_label=2)
print('AUC score :',metrics.auc(fpr, tpr))

f1_score 0.5408366051088168
precision_score 0.5615942028985508
recall_score 0.5652173913043478
AUC score : 0.7941176470588236
